In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision.models import resnet18

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self,in_c,out_c,kernel_size=3,stride=1,downsample=None):
        super(ResidualBlock,self).__init__()
        self.stride=stride
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_c,out_channels=out_c,stride=self.stride,kernel_size=kernel_size),
            nn.BatchNorm2d(out_c),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=in_c,out_channels=out_c,stride=self.stride,kernel_size=kernel_size),
            nn.BatchNorm2d(out_c)
        )
        self.downsample = downsample
    def forward(self,x):
        skip_connection = x
        x = self.conv1(x)
        x = self.conv2(x)
        if downsample:
            skip_connection=self.downsample(x)
        x = skip_connection+x
        x = nn.ReLU(x)
        return x

In [16]:
class Resnet(nn.Module):
    def __init__(self,res_block,block_repeat,num_classes=10):
        super(Resnet,self).__init__()
        self.res_block = res_block
        self.out_c = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=7,stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2)
        self.layer0 = self._make_layer(self.res_block, 64, block_repeat[0], stride = 1)
        self.layer1 = self._make_layer(self.res_block, 128, block_repeat[1], stride = 2)
        self.layer2 = self._make_layer(self.res_block, 256, block_repeat[2], stride = 2)
        self.layer3 = self._make_layer(self.res_block, 512, block_repeat[3], stride = 2)
        self.averagePool = nn.AvgPool2d(kernel_size=7,stride=1)
        self.fc = nn.Linear(in_features=512,out_features=num_classes)
   
    def _make_layer(self, block, out_c, blocks, stride=1):
            downsample = None
            if stride != 1 or self.out_c != out_c:
                downsample = nn.Sequential(
                    nn.Conv2d(self.out_c, out_c, kernel_size=1, stride=stride),
                    nn.BatchNorm2d(out_c),
                )
            layers = []
            layers.append(block(self.out_c, out_c, stride=stride, downsample=downsample))
            self.out_c = out_c
            for i in range(1, blocks):
                layers.append(block(self.out_c, out_c))
            return nn.Sequential(*layers)
            
    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.averagePool(x)
        x = self.fc(x)
        return x

In [17]:
model = Resnet(res_block=ResidualBlock,block_repeat=[2,2,2,2])

In [18]:
model


Resnet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequenti

In [14]:
torch_model = resnet18(num_classes=10)

In [15]:
torch_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  